In [12]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [19]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
DATASET_PATH = "/content/drive/MyDrive/cnn"


In [21]:
datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values
    validation_split=0.2,  # 20% of images used for validation
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

In [22]:
train_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(150, 150),
    batch_size=32,
    class_mode="categorical",
    subset="training"
)

Found 6778 images belonging to 2 classes.


In [23]:

val_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(150, 150),
    batch_size=32,
    class_mode="categorical",
    subset="validation"
)

Found 1694 images belonging to 2 classes.


In [24]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(len(train_generator.class_indices), activation="softmax")  # Output layer for classes
])



In [ ]:
# Step 5: Compile the Model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Step 6: Train the Model
history = model.fit(train_generator, validation_data=val_generator, epochs=10)

# Step 7: Evaluate Model
loss, acc = model.evaluate(val_generator)
print(f"Validation Accuracy: {acc*100:.2f}%")

# Step 8: Save Model
model.save("food_classification_model.h5")

# Step 9: Predict a New Image
from tensorflow.keras.preprocessing import image

def predict_image(image_path):
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)
    class_names = list(train_generator.class_indices.keys())

    print(f"Predicted Class: {class_names[predicted_class]}")

# Example usage
predict_image("/content/drive/MyDrive/cnn/food_test_c101_n1000_r64x64x1.h5")


In [ ]:
# history = model.fit(train_generator, validation_data=val_generator, epochs=10)
# It stores training progress (loss, accuracy, etc.) in the history object.

# history contains:
# history.history["loss"] → Training loss per epoch

# history.history["accuracy"] → Training accuracy per epoch

# history.history["val_loss"] → Validation loss per epoch

# history.history["val_accuracy"] → Validation accuracy per epoch


import matplotlib.pyplot as plt

# Plot accuracy
plt.plot(history.history["accuracy"], label="Train Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.legend()
plt.title("Model Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.show()

# Plot loss
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()
plt.title("Model Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()








